In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import missingno as msno
import matplotlib.pyplot as plt
#I had the same issue.

#There seems to be an issue with the source from which Plotly is loaded. Changing the notebook mode might help.

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from lightgbm import LGBMClassifier
import lightgbm as lgb
#import lazypredict
#from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

SEED = 42
fix_seed(SEED)

# Using Pandas to get familier with Your Data

we'll look at data recovered from the spaceship’s damaged computer system.
The train data is at the file path /kaggle/input/spaceship-titanic/train.csv.
We load and explore the data with the following commands:

In [ ]:
org_train = pd.read_csv(filepath_or_buffer='/kaggle/input/spaceship-titanic/train.csv')
org_train

# Interpreting Data Description
The results show 8 numbers for each column in your original dataset. The first number, the **count**,  shows how many rows have non-missing values.  

The second value is the **mean**, which is the average.  Under that, **std** is the standard deviation, which measures how numerically spread out the values are.

To interpret the **min**, **25%**, **50%**, **75%** and **max** values, imagine sorting each column from lowest to highest value.  The first (smallest) value is the min.  If you go a quarter way through the list, you'll find a number that is bigger than 25% of the values and smaller than 75% of the values.  That is the **25%** value (pronounced "25th percentile").  The 50th and 75th percentiles are defined analogously, and the **max** is the largest number.

In [ ]:
org_train.describe()

In [ ]:
org_train.isna().sum()

In [ ]:
org_train.nunique()

In [ ]:
org_test = pd.read_csv(filepath_or_buffer='/kaggle/input/spaceship-titanic/test.csv') # test data 
org_test

In [ ]:
org_test.describe()

In [ ]:
submission = pd.read_csv(filepath_or_buffer='/kaggle/input/spaceship-titanic/sample_submission.csv') # test data 
submission

In [ ]:
pd.DataFrame(org_train.isna().sum())


In [ ]:
org_train.nunique()

In [ ]:
def get_dataframe_info(df):
    """
    input
       df -> DataFrame
    output
       df_all_num_non_null -> DataFrame Info (sorted)
    """

    df_types = pd.DataFrame(df.dtypes)
    df_nulls = df.count()
    df_num_na = pd.DataFrame(df.isna().sum())
    df_num_unique = pd.DataFrame(df.nunique())
    
    df_null_count = pd.concat([df_types, df_nulls, df_num_na, df_num_unique], axis=1)
    df_null_count = df_null_count.reset_index()
    

    
    # Reassign column names
    col_names = ['features', 'types', 'num_non_null', 'num_null', 'num_unique']
    df_null_count.columns = col_names
    
    # Add this to sort
    df_all_num_non_null = df_null_count.sort_values(by=["types"], ascending=False)
    
    return df_all_num_non_null

get_dataframe_info(org_train)

In [ ]:
get_dataframe_info(org_test)

In [ ]:
pd.DataFrame(data=org_train.columns).T

In [ ]:
pd.DataFrame(data = org_train.isna().sum().sort_values(ascending= False), columns=['num_missing_data'])

In [ ]:
pd.DataFrame(data = org_test.nunique(), columns=['num_uniqe_data'])

# Label encodig
we change every categorical data to a number.That is each type will be subtuted by a number.for example we will substitute 1 for Grandmaster,2 for master ,3 for expert etc.. For implementing this we will first import Labelencoder from sklearn module.

Before getting into encoding,I will just breif you with types data variables present in this data:

Binary data : A binary variable a variable that has only 2 values..ie 0/1
Categorical data : A categorical variable is a variable that can take some limited number of values.for example,day of the week.It can be one of 1,2,3,4,5,6,7 only.
Ordinal data : An ordinal variable is a categorical variable that has some order associated with it.for example,the ratings that are given to a movie by a user.
Nominal data : Nominal value is a variable that has no numerical importance,such as occupation,person name etc..
Timeseries data : Time series data has a temporal value attached to it, so this would be something like a date or a time stamp that you can look for trends in time.

In [ ]:
encoded_train=pd.DataFrame()
encoded_test = pd.DataFrame()
label=LabelEncoder()
TARGET = 'Transported'
for column in  org_train.columns:
    if column == TARGET:
        encoded_train[column]=label.fit_transform(org_train[column])
    else:
        if (org_train[column].dtype=='object') and (org_test[column].dtype=='object'):
            encoded_train[column]=label.fit_transform(org_train[column])
            encoded_test[column]=label.fit_transform(org_test[column])
        else:
             encoded_train[column]=org_train[column]
             encoded_test[column]=org_test[column]  

In [ ]:
encoded_train.head(3)

In [ ]:
encoded_test.head(3)

In [ ]:
encoded_train.isnull().any()